# PremiseGen Transformer training
This trains the first transformer (premiseGen) which converts NL premises into RDF

In [2]:
import pandas as pd

# load data
train_path = r"../data/webnlg_train.csv"
eval_path = r"../data/webnlg_test.csv"


df = pd.read_csv(train_path, header=0, delimiter='\t')
df

df_eval = pd.read_csv(eval_path, header=0, delimiter='\t')

In [3]:
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df

df_eval = df_eval.sample(frac=1, random_state=42).reset_index(drop=True)

In [4]:
context_length = 100 # 100 chars context window should be big enough (?)

In [6]:
from local_gemma import LocalGemma2ForCausalLM
from transformers import AutoTokenizer

model = LocalGemma2ForCausalLM.from_pretrained("google/gemma-2-2b", preset="speed").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b")

# model_inputs = tokenizer("Turn Me On is a 35.1 minute long album produced by Wharton Tiers that was followed by the album entitled Take it Off. <T>", return_attention_mask=True, return_tensors="pt").to("cuda")
# generated_ids = model.generate(**model_inputs.to(model.device))
# 
# decoded_text = tokenizer.batch_decode(generated_ids)
# print(decoded_text)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [11]:
from datasets import Dataset
dataset = Dataset.from_pandas(df)

def tokenize_function(examples):
    examples["NL"] = tokenizer(examples["NL"], padding="max_length", return_attention_mask=True, return_tensors="pt").to("cuda")
    examples["Triples"] = tokenizer(examples["Triples"], padding="max_length", return_attention_mask=True, return_tensors="pt").to("cuda")
    return examples

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/35425 [00:00<?, ? examples/s]

Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.


ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

In [7]:
model_inputs = tokenizer("Hi", return_attention_mask=True, return_tensors="pt").to("cuda")
generated_ids = model.generate(**model_inputs.to(model.device))

decoded_text = tokenizer.batch_decode(generated_ids)
print(decoded_text)

D:\Code_Projects\Python\Logic3\.venv2\lib\site-packages\transformers\generation\utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


['<bos>Hi,\n\nI have a problem with the following code:\n\n<code>\n$sql = "']
